# EfficientNet

## Loading the Dataset

In [7]:
# set up access to Kaggle API
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


In [8]:
# download dataset from Kaggle and unzip file to store dataset in colab
!kaggle datasets download -d ibrahimfateen/wound-classification
!unzip wound-classification.zip

 96% 86.0M/89.8M [00:01<00:00, 102MB/s]
100% 89.8M/89.8M [00:01<00:00, 84.1MB/s]
Archive:  wound-classification.zip
  inflating: Wound_dataset copy/Abrasions/abrasions (1).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (10).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (11).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (12).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (13).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (14).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (15).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (16).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (17).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (18).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (19).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (2).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (20).jpg  
  inflating: Wound_dataset copy/Abrasions/abrasions (22).jpg  
  in

## Installing & Importing Necessary Libraries

In [1]:
!pip install efficientnet_pytorch
!pip install wandb

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=72c4a58b613e92ef49678164bff90690bf9dd255473ff7fc701ce80dfcc58fde
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.9/257.9 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00


In [10]:
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torch import nn, optim
from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [3]:
wandb.init(project="wound-classification", config={
    "learning_rate": 0.001,
    "architecture": "EfficientNet"
})

wandb: Currently logged in as: samyukta-palle (aipi549). Use `wandb login --relogin` to force relogin


## Preprocessing & Modelling

In [4]:
def preprocess(data_dir):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomApply([
            transforms.ColorJitter(brightness=0.2, contrast=0.1)], p=0.5),
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.RandomVerticalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    dataset = datasets.ImageFolder(data_dir, transform=transform)
    return dataset

In [13]:
def train(model, device, trainloader, valloader, criterion, optimizer, epochs, model_path):
    best_loss = float('inf')
    total_loss = []

    for epoch in range(epochs):
        print(f'Epoch [{epoch+1}/{epochs}]')
        model.train()
        running_loss = 0
        correct = 0
        total = 0

        progress_bar = tqdm(trainloader, desc=f'Epoch {epoch}/{epochs}', unit='batch')

        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            progress_bar.set_postfix(loss=running_loss / len(progress_bar), accuracy=100 * correct / total)

        train_loss = running_loss / len(trainloader)
        train_accuracy = 100 * correct / total

        # Validation
        val_loss, val_accuracy = validate(model, device, valloader, criterion)

        total_loss.append(train_loss)
        print(val_accuracy)
        # Log metrics to W&B
        wandb.log({"epoch": epoch+1, "train_loss": train_loss, "train_accuracy": train_accuracy,
                   "val_loss": val_loss, "val_accuracy": val_accuracy})

        # Save the model if the validation loss is the best so far
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), model_path)
            print("Saving the model")

    print('Training finished!')
    return total_loss

def validate(model, device, valloader, criterion):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    progress_bar = tqdm(valloader, desc='Validation', unit='batch')

    with torch.no_grad():
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            batch_loss = criterion(outputs, labels)
            val_loss += batch_loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            progress_bar.set_postfix(loss=val_loss / len(progress_bar), accuracy=100 * correct / total)

    val_loss /= len(valloader)
    val_accuracy = 100 * correct / total

    return val_loss, val_accuracy

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data_dir = '/content/Wound_dataset copy'
dataset = preprocess(data_dir)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
criterion = nn.CrossEntropyLoss()
epochs = 12

print(set(dataset.targets))
for fold, (train_index, val_index) in enumerate(skf.split(dataset.imgs, dataset.targets)):
    print(f'Fold [{fold + 1}/3]')

    train_dataset = torch.utils.data.Subset(dataset, train_index)
    val_dataset = torch.utils.data.Subset(dataset, val_index)

    trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

    model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=10)
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    wandb.init(project="wound-classification", name=f"fold_{fold}")

    model_path = f'/content/drive/MyDrive/saved_model_fold_{fold}.pth'
    train(model, device, trainloader, valloader, criterion, optimizer, epochs, model_path)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Fold [1/3]
Loaded pretrained weights for efficientnet-b0


Epoch [1/12]


Validation: 100%|██████████| 19/19 [00:05<00:00,  3.18batch/s, accuracy=70.2, loss=1]


70.23809523809524
Saving the model
Epoch [2/12]


Validation: 100%|██████████| 19/19 [00:05<00:00,  3.30batch/s, accuracy=77.7, loss=0.721]


...

92.00680272108843
Saving the model
Epoch [11/12]


Validation: 100%|██████████| 19/19 [00:06<00:00,  2.77batch/s, accuracy=95.1, loss=0.23]


95.06802721088435
Saving the model
Epoch [12/12]


Validation: 100%|██████████| 19/19 [00:06<00:00,  3.08batch/s, accuracy=94, loss=0.208]


94.04761904761905
Saving the model
Training finished!


## Plots

Train & Validation Accuracy; Train & Validation Loss is in W&B graphs

In [4]:
# download dataset from Kaggle and unzip file to store dataset in colab
!kaggle datasets download -d yasinpratomo/wound-dataset
!unzip wound-dataset.zip

100% 13.9M/13.9M [00:00<00:00, 73.6MB/s]
100% 13.9M/13.9M [00:00<00:00, 69.7MB/s]
Archive:  wound-dataset.zip
  inflating: Wound_dataset/Abrasions/abrasions (1).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (10).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (11).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (12).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (13).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (14).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (15).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (16).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (17).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (18).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (19).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (2).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (20).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (21).jpg  
  inflating: Wound_dataset/Abrasions/abrasions (22).jpg  
  inflating: Wound_dat

In [7]:
import torch
import numpy as np
from sklearn.metrics import confusion_matrix
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet


# Define the path to the saved model file
model_path = '/content/model.pth'

# Load the model architecture
model = EfficientNet.from_name('efficientnet-b0', num_classes=10)

# Load the model's weights from the saved file
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model.eval()

# Define the transformation for the validation dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the validation dataset
test_dataset = datasets.ImageFolder('/content/Wound_dataset', transform=transform)

# Create a DataLoader for the validation dataset
test_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Initialize lists to store actual and predicted labels
all_labels = []
all_preds = []

# Iterate over the validation dataset and make predictions
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()  # Assuming you're using GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Create the confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

print("Confusion Matrix:")
print(conf_matrix)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.